In [ ]:
from random import seed
from random import random
import numpy as np

In [ ]:
# First step is to initialize simple ANN with layers organized as arrays of dictionaries
# We start with one hidden layers for the beginning

def ANN(n_inputs, n_hidden, n_outputs):
    
    network = list()
    
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]

    network.append(hidden_layer)
    
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    
    network.append(output_layer)
    
    return network

In [ ]:
seed(1)
network = ANN(3, 2, 3)
print(network)

In [ ]:
for layer in network:
    
    for neuron in layer:
        
        print(neuron['weights'])

In [ ]:
# Forward propagate
'To calculate an output from a neural network by propagating an input signal through each layer until the output layer outputs its values. We call this forward-propagation. Forward prop can be separated into three parts: 1) Neuron activation, 2) Neuron Transfer, 3) Forward Propagation'

In [ ]:
# Neuron Activation
'The first step is to calculate the activation of one neuron given an input.'

def activate(w, inputs):
    
    activation = w[-1] # bias
    
    for i in range(len(w)-1):
        
        activation += w[i]*inputs[i] # computing sum over all products of weights multiplied by input  
    
        return activation
    

In [ ]:
#Non-linearity  we are going to use is ReLU

def ReLu(activation):
    
    return activation *(activation > 0) 


In [ ]:
# Forward pass
'We need to calculate an output from an NN by propagating an input signal through each layer until the output layer'

def forward_pass(network,inputs):
    
    for layer in network:
        
        input_output = []
        
        for neuron in layer:
            
            activation = activate(neuron['weights'], inputs)
            
            neuron['output'] = ReLu(activation)
            
            input_output.append(neuron['output'])
            
        inputs =  input_output
        
    return inputs 
               

In [ ]:
print(network)

In [ ]:
inputs = [3,0,None]
output = forward_pass(network,inputs)
print(output)

In [ ]:
print(network[1])

In [ ]:
for layer in network:
    
    for neuron in layer:
        
        print(neuron['output'])

In [ ]:
# BackProp
'Our goal is to backpropagate error (to update weights) between the expected and real(the one that is obtained during the forward pass) outputs.We start with computing derivative of the activation'

In [ ]:
def ReLuDerivative(x):
    
    return np.greater(x, 0).astype(int)
    

In [ ]:
#Error backprop
'Since we going back, we start error calculation from the last layer'

def back_prop_error(network,expected):
    
    for i in reversed(range(len(network))):
        
        layer = network[i]
        
        errors = list()
        
        # Go here in the the second iteration
        
        if i !=len(network)-1:
            
            for j in range(len(layer)):
                
                error = 0.0
                
                for neuron in network[i+1]:
                    
                    error += (neuron['weights'][j]*neuron['delta'])
                    
                errors.append(error)
          
        # During the first iteration we compute error on the last (output) layer of ANN
        
        else:
            
            for j in range(len(layer)):
                
                neuron = layer[j]
                
                errors.append(expected[j] - neuron['output'])
        
        # Move an error 'back' further to the left (closer to the input)        
        
        for j in range(len(layer)):
            
            neuron = layer[j]
            neuron['delta'] = errors[j]*ReLuDerivative(neuron['output'])
            
            

In [ ]:
network

In [ ]:
'Now the next step is to train ANN (we need to update weights by applying gradient descent algorithm'

In [ ]:
# We start with updating weights acc to GD: 
# weight = weight + learning_rate * error * input

def weights_update(network, inputs , learning_rate):
    
    for i in range(len(network)):
        
        inputs = inputs[:-1] # Return all exept last one (bias)
        
        if i!=0:
            
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in network[i]:
            
            for j in range(len(inputs)):
                
                neuron['weights'][j] += learning_rate * neuron['delta'] * inputs[j]
                
            neuron['weights'][-1] += learning_rate * neuron['delta']
               

In [ ]:
# To train Network

'We need to make one forward pass, then to compute our error between the real value and what we actually expected (supervise learning approach).  This difference is reflected in a cost function with we are going to optimize by using Gradient Descent.'

def train_network(network,dataset,learning_rate, epochs, n_outputs):
    
    for epoch in range(epochs):
        
        cost = 0
        
        for row in dataset:
            
            outputs = forward_pass(network,row)
            expected = [0 for i in range(n_outputs)] # Define expected class labels
            expected[row[-1]] = 1
            
            cost += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) #MSE
            
            back_prop_error(network,expected)
            
            weights_update(network,row,learning_rate)
            
        print('>epoch=%d, lrate=%.3f, cost=%.3f' % (epoch, learning_rate, cost))

In [ ]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

In [ ]:
inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset])) # Binary problem, therefore only two outputs
learning_rate = 1e-4
epochs = 50

In [ ]:
network = ANN(inputs,2,n_outputs)
network

In [ ]:
train_network(network,dataset,learning_rate,epochs,n_outputs)
for layer in network:
	print(layer)